In [15]:
# from google.colab import auth
# auth.authenticate_user()
from langchain.agents import create_sql_agent 
from langchain.agents.agent_toolkits import SQLDatabaseToolkit 
from langchain.sql_database import SQLDatabase 
from langchain.llms.openai import OpenAI 
from langchain.agents import AgentExecutor 
from langchain.agents.agent_types import AgentType
from langchain.chat_models import ChatOpenAI, ChatVertexAI
from google.cloud import aiplatform
from google.cloud.sql.connector import Connector
import config

In [ ]:
#setup & init env vars
PROJECT_ID="jared-playground"
REGION="us-central1"
INSTANCE_NAME="gen-ai-text-to-mysql"
INSTANCE_CONNECTION_NAME = f"{PROJECT_ID}:{REGION}:{INSTANCE_NAME}" # i.e demo-project:us-central1:demo-instance
DB_USER = "root"
DB_NAME = "classicmodels"

In [ ]:
#Just double check stuff....
print(f"Your instance connection name is: {INSTANCE_CONNECTION_NAME}")

!gcloud config set project $PROJECT_ID
!gcloud config get project
!gcloud auth list

In [12]:
def get_db():
    # initialize Connector object
    connector = Connector()
    conn = connector.connect(
        INSTANCE_CONNECTION_NAME,
        "pymysql",
        user=DB_USER,
        db=DB_NAME
    )
    db = SQLDatabase.from_uri(
        "mysql+pymysql://",
        engine_args = {'creator': getconn},
    )
    return db

In [13]:
def test_sql_agent(db, llm, question):
    
    toolkit = SQLDatabaseToolkit(db=db, llm=llm)
    agent_executor = create_sql_agent(
        llm=llm,
        toolkit=toolkit,
        verbose=True,
        # agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    )
    agent_executor.run(question)

In [14]:
test_sql_agent(
    db = get_db(),
    llm = VertexAI(model_name="code-bison@001", temperature=0, verbose=True),
    question = "How many orders were placed in the year 2005?"
)



> Entering new AgentExecutor chain...
Question: How many orders were placed in the year 2005?
Thought: I should look at the tables in the database to see what I can query.  Then I should query the schema of the most relevant tables.
Action: sql_db_list_tables
Action Input: 
Observation: customers, employees, offices, orderdetails, orders, payments, productlines, products
Thought:The tables orders and orderdetails seem relevant.  I will query the schema for those tables.
Action: sql_db_schema
Action Input: orders, orderdetails
Observation: 
CREATE TABLE orderdetails (
	`orderNumber` INTEGER NOT NULL, 
	`productCode` VARCHAR(15) NOT NULL, 
	`quantityOrdered` INTEGER NOT NULL, 
	`priceEach` DECIMAL(10, 2) NOT NULL, 
	`orderLineNumber` SMALLINT NOT NULL, 
	PRIMARY KEY (`orderNumber`, `productCode`), 
	CONSTRAINT orderdetails_ibfk_1 FOREIGN KEY(`orderNumber`) REFERENCES orders (`orderNumber`), 
	CONSTRAINT orderdetails_ibfk_2 FOREIGN KEY(`productCode`) REFERENCES products (`productCode`)


In [16]:
test_sql_agent(
    db = get_db(),
    llm = OpenAI(model_name="text-davinci-003", temperature=0, verbose=True, openai_api_key=config.openai_api_key),
    question = "How many orders were placed in the year 2005?"
)



> Entering new AgentExecutor chain...
Thought: I should use the sql_db_list_tables tool to get a list of tables in the database.
Action: sql_db_list_tables
Action Input: ""
Observation: customers, employees, offices, orderdetails, orders, payments, productlines, products
Thought: The orders table looks like it might have the information I need.
Thought: I should use the sql_db_schema tool to get the schema of the orders table.
Action: sql_db_schema
Action Input: orders
Observation: 
CREATE TABLE orders (
	`orderNumber` INTEGER NOT NULL, 
	`orderDate` DATE NOT NULL, 
	`requiredDate` DATE NOT NULL, 
	`shippedDate` DATE, 
	status VARCHAR(15) NOT NULL, 
	comments TEXT, 
	`customerNumber` INTEGER NOT NULL, 
	PRIMARY KEY (`orderNumber`), 
	CONSTRAINT orders_ibfk_1 FOREIGN KEY(`customerNumber`) REFERENCES customers (`customerNumber`)
)ENGINE=InnoDB DEFAULT CHARSET=latin1

/*
3 rows from orders table:
orderNumber	orderDate	requiredDate	shippedDate	status	comments	customerNumber
10100	2003-01